In [ ]:
import matplotlib.pyplot as plt
import os
from tiled.client import from_uri
from reduction import *
from conversions import *
from curve_fitting import *
from file_handling import read_reduction_tiled
from pipeline import horizontal_cut_automatic_fit
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, peak_prominences, argrelmin
from astropy.modeling import models

# Definition of Constants

In [ ]:
# Initialize the Tiled server
TILED_URI = os.getenv("TILED_URI")
TILED_API_KEY = os.getenv("TILED_API_KEY")

tiled_client = from_uri(TILED_URI, api_key=TILED_API_KEY)
TILED_BASE_URI = tiled_client.uri

In [ ]:
# 2pi / (nanometer in real-space * conversion factor between nano meter and Angstrom)
q_target = 2 * np.pi / (30) * 0.1
print("Q-target", q_target)

In [ ]:
parameters_horizontal_2024_10_17 = {
    "input_uri_data": "",
    "input_uri_mask": "processed/masks/2024_10_17_GISAXS_YW_inverted",
    "beamcenter_x": 691.416,
    "beamcenter_y": 1280.79,
    "incident_angle": 0.12,
    "sample_detector_dist": 3569.32,
    "wavelength": 1.2398,
    "pix_size": 172,
    "cut_half_width": 5,
    "cut_pos_y": 1280 - 105,
    "x_min": 691 - 250,
    "x_max": 691 + 250,
    "output_unit": "q",
}
parameters_horizontal_2024_10_23 = {
    "input_uri_data": "",
    "input_uri_mask": "processed/masks/2024_10_17_GISAXS_YW_inverted",
    "beamcenter_x": 691.007,
    "beamcenter_y": 1281.21,
    "incident_angle": 0.12,
    "sample_detector_dist": 3590.48,
    "wavelength": 1.2398,
    "pix_size": 172,
    "cut_half_width": 5,
    "cut_pos_y": 1176,
    "x_min": 391,
    "x_max": 991,
    "output_unit": "q",
}

# Selection of a subnode in Tiled with several scans to process

In [ ]:
experiment_root = "raw/2024_10_23/New_Batch"
# experiment_root = "raw/2024_10_16/GISAXS/34K_58K_static_data/"
# experiment_root = "raw/2024_10_16/GISAXS/Randomized"
# experiment_root = "raw/2024_10_16/GISAXS/34K_58K_static_data"
# experiment_root = "raw/ALS-S2VP58k"
experiment_root_client = from_uri(
    f"{TILED_BASE_URI}/{experiment_root}", api_key=TILED_API_KEY
)
scans = [
    scan
    for scan in experiment_root_client
    if "sfloat" in scan and "autoexpose" not in scan
]
print("Number of scans in ", experiment_root, ": ", len(scans))
print("\n".join(f"{index}: {entry}" for index, entry in enumerate(scans)))

# Step by Step Processing for a single scan

In [ ]:
selected_scan_id = 0
selected_scan_name = scans[selected_scan_id]
input_uri_data = experiment_root_client[selected_scan_name].uri
input_uri_data = input_uri_data.replace(TILED_BASE_URI, "")
print(input_uri_data)
parameters_horizontal_2024_10_23["input_uri_data"] = input_uri_data
horizontal_cut_automatic_fit(**parameters_horizontal_2024_10_23)

In [ ]:
output_uri = input_uri_data.replace("raw", "processed")
parts = output_uri.split("/")
reduced_uri = f"{output_uri}/{parts[-1]}_horizontal-cut"
peak_uri = f"{output_uri}/{parts[-1]}_fitted-peaks"
print(reduced_uri)
fitted_peaks = from_uri(f"{TILED_BASE_URI}/{peak_uri}", api_key=TILED_API_KEY)

In [ ]:
df = fitted_peaks.read()
peak_means = df["x"].values.tolist()
peak_amplitudes = df["y"].values.tolist()
peak_fwhm = df["fwhm"].values.tolist()
peak_stddevs = [fwhm / (2 * math.sqrt(2 * math.log(2))) for fwhm in peak_fwhm]
baseline_removal = "zhang"
fit_range_min = 0.005
fit_range_max = 0.035

q, intensity = read_reduction_tiled(
    reduced_uri,
    fit_range=[fit_range_min, fit_range_max],
    baseline_removal=None,
)
# New fit range
if baseline_removal == "linear":
    first_inflection_point = argrelmin(intensity)[0]
    print("First inflection point", q[first_inflection_point[0]])
    fit_range_min = q[first_inflection_point[0]]
q_without_baseline, intensity_without_baseline = read_reduction_tiled(
    reduced_uri,
    fit_range=[fit_range_min, fit_range_max],
    baseline_removal=baseline_removal,
)

individual_models = []
for ii, (mean, amplitude, fwhm) in enumerate(
    zip(peak_means, peak_amplitudes, peak_fwhm)
):
    stddev = fwhm / (2 * math.sqrt(2 * math.log(2)))
    print("Peak position", mean, amplitude, "Full-width-half-max", fwhm)
    if ii == 0:
        sum_models = models.Gaussian1D(amplitude=amplitude, mean=mean, stddev=stddev)
    else:
        g = models.Gaussian1D(amplitude=amplitude, mean=mean, stddev=stddev)
        individual_models.append(g)
        sum_models += g
fitted_intensity = sum_models(q)

In [ ]:
plt.figure()
plt.plot(q, intensity, label="")
plt.plot(q_without_baseline, intensity_without_baseline, label="Baseline Removed")
scan_name = reduced_uri.split("/")[-1]
plt.title(reduced_uri.split("/")[-1])
plt.plot(q, fitted_intensity)
intensity_at_closest_q = fitted_intensity[np.argmin(abs(q - peak_means[1]))]
plt.vlines(
    x=peak_means[1],
    ymin=intensity_at_closest_q - peak_amplitudes[1] / 2,
    ymax=intensity_at_closest_q,
    color="red",
)
# Plot the full-width at half-maximum
plt.hlines(
    y=intensity_at_closest_q - peak_amplitudes[1] / 2,
    xmin=peak_means[1] - peak_fwhm[1] / 2,
    xmax=peak_means[1] + peak_fwhm[1] / 2,
    color="red",
    linestyle="--",
)
plt.annotate(
    f"Peak Position: {peak_means[1]:.5}",
    xytext=[peak_means[1], np.max(intensity)],
    xy=[peak_means[1], peak_amplitudes[1]],
    arrowprops=dict(arrowstyle="->"),
)
plt.annotate(
    f"FWHM: {peak_fwhm[1]:.7}",
    xy=[peak_means[1], intensity_at_closest_q - peak_amplitudes[1] / 1.5],
)
plt.savefig(f"plots/{scan_name}.png")

In [ ]:
fit_range_min = max(0, q_target - 0.015)
fit_range_max = q_target + 0.015
baseline_removal = "zhang"
# baseline_removal = None
automatic_peak_fit_tiled(
    input_uri_raw=reduced_uri,
    reduction_type="horizontal-cut",
    fit_range=[fit_range_min, fit_range_max],
    baseline_removal=baseline_removal,
)

# Iteration over all scans in the selected sub node

In [ ]:
for selected_scan_id in range(len(scans)):
    selected_scan_name = scans[selected_scan_id]
    input_uri_data = experiment_root_client[selected_scan_name].uri
    input_uri_data = input_uri_data.replace(TILED_BASE_URI, "")
    print(input_uri_data)
    if "10_23" in input_uri_data:
        parameters_horizontal_2024_10_23["input_uri_data"] = input_uri_data
        horizontal_cut_automatic_fit(**parameters_horizontal_2024_10_23)
    else:
        parameters_horizontal_2024_10_17["input_uri_data"] = input_uri_data
        horizontal_cut_automatic_fit(**parameters_horizontal_2024_10_17)

    output_uri = input_uri_data.replace("raw", "processed")
    parts = output_uri.split("/")
    reduced_uri = f"{output_uri}/{parts[-1]}_horizontal-cut"
    peak_uri = f"{output_uri}/{parts[-1]}_fitted-peaks"
    print(reduced_uri)
    fitted_peaks = from_uri(f"{TILED_BASE_URI}/{peak_uri}", api_key=TILED_API_KEY)

    df = fitted_peaks.read()
    peak_means = df["x"].values.tolist()
    peak_amplitudes = df["y"].values.tolist()
    peak_fwhm = df["fwhm"].values.tolist()
    peak_stddevs = [fwhm / (2 * math.sqrt(2 * math.log(2))) for fwhm in peak_fwhm]
    baseline_removal = "zhang"
    fit_range_min = 0.005
    fit_range_max = 0.035

    q, intensity = read_reduction_tiled(
        reduced_uri,
        fit_range=[fit_range_min, fit_range_max],
        baseline_removal=None,
    )
    # New fit range
    if baseline_removal == "linear":
        first_inflection_point = argrelmin(intensity)[0]
        print("First inflection point", q[first_inflection_point[0]])
        fit_range_min = q[first_inflection_point[0]]
    q_without_baseline, intensity_without_baseline = read_reduction_tiled(
        reduced_uri,
        fit_range=[fit_range_min, fit_range_max],
        baseline_removal=baseline_removal,
    )

    individual_models = []
    for ii, (mean, amplitude, fwhm) in enumerate(
        zip(peak_means, peak_amplitudes, peak_fwhm)
    ):
        stddev = fwhm / (2 * math.sqrt(2 * math.log(2)))
        print("Peak position", mean, amplitude, "Full-width-half-max", fwhm)
        if ii == 0:
            sum_models = models.Gaussian1D(
                amplitude=amplitude, mean=mean, stddev=stddev
            )
        else:
            g = models.Gaussian1D(amplitude=amplitude, mean=mean, stddev=stddev)
            individual_models.append(g)
            sum_models += g
    fitted_intensity = sum_models(q)

    plt.figure()
    plt.plot(q, intensity, label="")
    plt.plot(q_without_baseline, intensity_without_baseline, label="Baseline Removed")
    scan_name = reduced_uri.split("/")[-1]
    plt.title(reduced_uri.split("/")[-1])
    plt.plot(q, fitted_intensity)
    intensity_at_closest_q = fitted_intensity[np.argmin(abs(q - peak_means[1]))]
    plt.vlines(
        x=peak_means[1],
        ymin=intensity_at_closest_q - peak_amplitudes[1] / 2,
        ymax=intensity_at_closest_q,
        color="red",
    )
    # Plot the full-width at half-maximum
    plt.hlines(
        y=intensity_at_closest_q - peak_amplitudes[1] / 2,
        xmin=peak_means[1] - peak_fwhm[1] / 2,
        xmax=peak_means[1] + peak_fwhm[1] / 2,
        color="red",
        linestyle="--",
    )
    plt.annotate(
        f"Peak Position: {peak_means[1]:.5}",
        xytext=[peak_means[1], np.max(intensity)],
        xy=[peak_means[1], peak_amplitudes[1]],
        arrowprops=dict(arrowstyle="->"),
    )
    plt.annotate(
        f"FWHM: {peak_fwhm[1]:.7}",
        xy=[peak_means[1], intensity_at_closest_q - peak_amplitudes[1] / 1.5],
    )
    plt.savefig(f"plots/{scan_name}.png")

# Adding newly processed samples and their peak information to the experiment summary

In [ ]:
from tiled.utils import path_from_uri

client = from_uri(TILED_URI, api_key=TILED_API_KEY, include_data_sources=True)
container_client = client["processed"]["2024_10_17-23_58k-34k_mixture"]
csv_client = container_client["2024_10_17-23_58k-34k_mixture"]
csv_file_local_uri = path_from_uri(
    csv_client.data_sources()[0]["assets"][0]["data_uri"]
)
df_original = pd.read_csv(csv_file_local_uri)
# data.to_csv(csv_file_local_uri, index=False)
print(df_original.columns)

In [ ]:
experiment_root = "processed/2024_10_23/New_Batch"
experiment_root_client = from_uri(
    f"{TILED_BASE_URI}/{experiment_root}", api_key=TILED_API_KEY
)
df_new_samples = pd.DataFrame(columns=df_original.columns)
for scan_key in experiment_root_client.keys():
    scan_client = experiment_root_client[scan_key]
    for reduction_key in scan_client:
        if "peak" in reduction_key:
            try:
                peak_info = scan_client[reduction_key].read()
                peak_position = peak_info["x"].values[1]
                fwhm = peak_info["fwhm"].values[1]
                print(fwhm)
                new_row = pd.DataFrame(
                    {
                        "Scan Key": [scan_key],
                        "Peak Position": [peak_position],
                        "Fwhm": [fwhm],
                        "Domain Spacing (nm)": [0.1 * 2 * np.pi / peak_position],
                        "Grain Size": [0.1 * 2 * np.pi / fwhm],
                    }
                )
                df_new_samples = pd.concat([df_new_samples, new_row], ignore_index=True)
            except Exception as e:
                print(e)
                pass

In [ ]:
df_new_samples

In [ ]:
df_combined = pd.concat([df_original, df_new_samples], ignore_index=True)

In [ ]:
df_combined.to_csv(csv_file_local_uri)

# Testing of peak fitting by itself for a selected scan

In [ ]:
selected_scan_id = 0
selected_scan_name = scans[selected_scan_id]
reduced_uri = experiment_root_client[
    f"{selected_scan_name}/{selected_scan_name}_horizontal-cut"
].uri
reduced_uri = reduced_uri.replace(TILED_BASE_URI, "")

fit_range_min = max(0, q_target - 0.015)
fit_range_max = q_target + 0.015
q, intensity = read_reduction_tiled(
    reduced_uri,
    fit_range=[fit_range_min, fit_range_max],
    baseline_removal=None,
)
plt.plot(q, intensity)
first_inflection_point = argrelmin(intensity)[0]
print("First inflection point", q[first_inflection_point[0]])
fit_range_min = q[first_inflection_point[0]]
plt.axvline(fit_range_min, color="red")
peak_indices = find_peaks(intensity)[0]
prominences = peak_prominences(intensity, peak_indices)
print("Peak indices", peak_indices)
print("Prominences", prominences)
# Edit this peak number
peak_number = np.argmax(prominences[0])
peak_location = q[peak_indices[peak_number]]
print("Peak location", peak_location)
plt.axvline(peak_location, color="red")
baseline_removal = "modpoly"
q_subset, intensity_subset = read_reduction_tiled(
    reduced_uri,
    fit_range=[fit_range_min, fit_range_max],
    baseline_removal=baseline_removal,
)
plt.plot(q_subset, intensity_subset, label="Baseline removed")

In [ ]:
parameters_fitting = {
    "input_uri_reduction": reduced_uri,
    "x_peaks": [fit_range_min, peak_location],
    "y_peaks": [
        intensity[np.argmin(abs(q - fit_range_min))],
        intensity[np.argmin(abs(q - peak_location))],
    ],
    "stddevs": [0.00001, 0.003],
    "fwhm_Gs": [0.001, 0.001],
    "fwhm_Ls": [0.001, 0.001],
    "peak_shape": "gaussian",
    "fit_range": [fit_range_min, fit_range_max],
    "baseline_removal": baseline_removal,
}
print(parameters_fitting)
simple_peak_fit_tiled(**parameters_fitting)